In [1]:
#%matplotlib widget
#DESIGN PARAMETERS FOR NEURAL NETWORK
NR_LSTM_UNITS = 2 
IMAGE_INPUT_SIZE_W = 640
IMAGE_INPUT_SIZE_H = 480

IMAGE_AFTER_CONV_SIZE_W = 18
IMAGE_AFTER_CONV_SIZE_H = 13
#for 3x3 kernels, n=num_layers: len_in = 2^n*len_out + sum[i=1..n](2^i)
#CONV_LAYER_LENGTH = 5

# LSTM_IO_SIZE = 18*13
# LSTM_HIDDEN_SIZE = 18*13
CONV_OUT = 100

RGB_CHANNELS = 3
TIMESTEPS = 10 # size videos
BATCH_SIZE = 4
SLICE_FROM_TIMESTEP = 1 #slices from timestep SLICE_FROM_TIMESTEP to the last one

NR_EPOCHS = 10

VALIDATION_SPLIT = 0.0 #indicated ratio of training to validation data: 0.2 -> 20% VALIDATION data
RANDOMIZED_SEED = 20
SHUFFLE_DATASET = False

learning_rate = 0.001 # reduce factos of 10 .. some epoch later.
momentum = 0.9
print("Hyperparameters defined")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms, utils

import numpy as np
#import memory_profiler
import time
import os
import os.path as path
import copy
# from torch.utils.data.sampler import SequentialSampler


from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer

import matplotlib.pyplot as plt
plt.switch_backend('Qt4Agg')
#import matplotlib.gridspec as grd

print("Imports done")

#change configuration in settings.py
crop_im_size = [186,250]
c = Config()
# c.config['indexing']['grouping'] = 'person_sequence_angle'
# c.config['transformers']['DimensionResize']['dimension'] = TIMESTEPS
# #c.config['indexing']['people selection'] = [1]
# #c.config['indexing']['sequences_selection'] = ['nm-01']
# c.config['pose']['load'] = False
# c.config['flow']['load'] = False
# c.config['heatmaps']['load'] = False
# #c.config['scenes']['sequences'] = ['nm']
# #c.config['scenes']['angles'] = ['108']
# c.config['dataset_output'] = {
# #         'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
#         'data': ['scenes'],
#         'label': "annotations"}
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)

Hyperparameters defined
Imports done
loading configuration  flows_downscale
[OK]


In [2]:
class TEST_CNN_LSTM(nn.Module):
    def __init__(self):
        super(TEST_CNN_LSTM, self).__init__()
        self.avialable_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        #input 382x382
        self.conv1 = nn.Conv2d(30,60,3) #output 380x380
        self.pool1 = nn.MaxPool2d(2,2) #output 190x190
        self.conv2 = nn.Conv2d(60,30,3) # output 188x188
        self.pool2 = nn.MaxPool2d(2,2) # output 94x94
        self.conv3 = nn.Conv2d(30,1,3) # output 92x92
        self.pool3 = nn.MaxPool2d(2,2) # output 46x46
#         self.conv4 = nn.Conv2d(30,1,3)  # output 44x44
#         self.pool4 = nn.MaxPool2d(2,2) #  output 22x22
#         self.conv5 = nn.Conv2d(30,1,3)  # output 20x20
#         self.pool5 = nn.MaxPool2d(2,2) # output 10x10
        
        self.fc1 = nn.Linear(CONV_OUT,120)
        self.fc2 = nn.Linear(120,20)
        self.fc3 = nn.Linear(20,3)
        
    def forward(self,x):
#         print("Input list len:",len(x))
#         print("Input elemens size:", x.size())
        x = x.float()
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
#         x = self.pool4(F.relu(self.conv4(x)))
#         x = self.pool5(F.relu(self.conv5(x)))
#         print("Inputs after conv:", x.size())
        x = x.view(BATCH_SIZE,CONV_OUT) #output.view(seq_len, batch, num_dir*hidden_size)
#         x = torch.squeeze(x)
#         print("Input before FC:", x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
#         print("Input after FC:", x.size())
        #x = x.squeeze(1)
#         x = x.permute(1,2,0)
        #print ("Size network output", x.shape)
        return x
print("Network class defined")

test_net = TEST_CNN_LSTM()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
test_net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(test_net.parameters(), lr=learning_rate, momentum=momentum)
print("Neural network defined")
print("Input: 94x94, 3 conv layer")

Network class defined
cpu
Neural network defined
Input: 94x94, 3 conv layer


In [3]:
dataset_size = len(dataset)
indices = list(range(dataset_size))
print("Dataset size:", len(dataset))
print("Indices size:", len(indices))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
print("Split:", split)
if SHUFFLE_DATASET:
    np.random.seed(RANDOMIZED_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = torch.utils.data.SequentialSampler(train_indices)
test_sampler = torch.utils.data.SequentialSampler(test_indices)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)
print("Training data loaded")

Dataset size: 24
Indices size: 24
Split: 0
Training data loaded


In [4]:
parameters_conv1_init = list(test_net.conv1.parameters())
bias_conv1_init = parameters_conv1_init[1].detach().numpy()
weights_conv1_init = parameters_conv1_init[0].detach().numpy().flatten()

parameters_fc1_init = list(test_net.fc1.parameters())
bias_fc1_init = parameters_fc1_init[1].detach().numpy()
weights_fc1_init = parameters_fc1_init[0].detach().numpy().flatten()

print("FC1: Mean",np.mean(weights_fc1_init),"Variance",np.var(weights_fc1_init))
print("Saved initial weights and biases from net")

FC1: Mean 0.0011104866 Variance 0.0033620333
Saved initial weights and biases from net


In [ ]:
def plot_input(data_in):
    if BATCH_SIZE == 1:
        print("First input:")
        fig,axs  = plt.subplots(nrows=1, ncols=10,figsize=(len(data_in),1))
        for item,ax in zip(data_in,axs):
            #item.imshow(data_in.permute(1,2,0))
            #print(item.squeeze(0))
            ax.imshow(item.squeeze(0).permute(2,1,0))
        #plt.show()
    else: 
        print("Batch size not suitable for plot")
print("Plot function defined")
# plot_input(images_in_first)

Plot function defined


In [ ]:
def check_input_labels(dataset):
    print("Number elements:", len(dataset))
    print("Batch size:",BATCH_SIZE)
    print("Containing labels:")
    for dataset_item in dataset:
        _, labels = dataset_item
        print(labels)
#check_input_labels(dataset)

### Test set and evaluate computing time etc.

In [ ]:
#PREPARATION FOR TRAINING
loss_array = []
learning_rate_array = []

print('Start training...')
print("Expected loss with {} different classes and {} data elements: {}".format(3, len(dataset)-split, (len(dataset)-split)*np.log(3)))
running_loss = 0.0
#print("Data set length:", len((train_loader)), "Validation length:", len(test_loader))
print("Batch size:", BATCH_SIZE)
print("Evaluating first element...")
start_time = time.time()
i, batch = next(iter(enumerate(train_loader)))
inputs, labels = batch
images_in_first = inputs
# data_in = [s.to(device) for s in inputs['flows']]
data_in = inputs['flows'].to(device)
labels = labels.to(device)
# print("Time steps:{}, input sequence length:{}".format(TIMESTEPS,len(data_in)))
# print("NN input: ",len(data_in),len(data_in[0]),len(data_in[0][0]),len(data_in[0][0][0]),len(data_in[0][0][0][0]))
optimizer.zero_grad() 
#


# plot_input(images_in_first)
# print("Data in [-1]:")
# print(data_in[-1].shape)
# print("Data in pure:")
# print("List with elementes " ,len(data_in), "Obj", data_in[0].size())
# print("Input length:", (data_in.size()))
outputs = test_net(data_in)
print("Expected output format: [BATCH, NR_CLASSES, TIMESTEPS]")
print("Output format:", outputs.size())
print("Expected label format: [BATCH, TIMESTEPS] (with int-label as each element indicating the correct one)")
print("Labels:", len(labels), labels.size())
print("Labels content:", labels)
print("Output and Label sizes:",outputs.shape, labels.shape)
print("Output and Label:", outputs, labels)
loss = criterion(outputs.float(),labels.long())
loss.backward() 
optimizer.step()
running_loss += loss.data.item()
elapsed_time = time.time() - start_time;
#loss_array.append(running_loss)
#learning_rate_array.append(learning_rate)
print("Loss:{}, expected loss:{}".format(running_loss, np.log(3)))
print("Time needed:{}s".format(elapsed_time))
print("Expected loss for total training data: ", (len(dataset)-split)*np.log(3))
print("Expected training time per epoch:{} min".format(elapsed_time* len(train_loader)/60))
print("Estimated total training time:{} hours".format(elapsed_time* len(train_loader)*NR_EPOCHS/3600))

Start training...
Expected loss with 3 different classes and 24 data elements: 26.366694928034633
Batch size: 4
Evaluating first element...
Expected output format: [BATCH, NR_CLASSES, TIMESTEPS]
Output format: torch.Size([4, 3])
Expected label format: [BATCH, TIMESTEPS] (with int-label as each element indicating the correct one)
Labels: 4 torch.Size([4])
Labels content: tensor([1, 1, 1, 1])
Output and Label sizes: torch.Size([4, 3]) torch.Size([4])
Output and Label: tensor([[ 0.4782, -1.4257, -1.5516],
        [ 0.5177, -1.4276, -1.6040],
        [ 0.5544, -1.3932, -1.6783],
        [ 0.5732, -1.4071, -1.7052]], grad_fn=<AddmmBackward>) tensor([1, 1, 1, 1])
Loss:2.1740198135375977, expected loss:1.0986122886681098
Time needed:17.94371271133423s
Expected loss for total training data:  26.366694928034633
Expected training time per epoch:1.7943712711334228 min
Estimated total training time:0.29906187852223715 hours


In [ ]:
# plt.show()

### Start Training here


In [ ]:
for epoch in range(NR_EPOCHS): 
    running_loss = 0.0
    start_time = time.time()
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        data_in = inputs['flows'].to(device)
        labels = labels.to(device)
#         print("Labels",labels.size())
            
        outputs = test_net(data_in)
        loss = criterion(outputs.float(),labels.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data.item()
        
#         total = 0;
#         correct = 0;
#         for item in labels:
#             labels = torch.tensor([[item]])
#         _, predicted = torch.max(outputs.data, 1)
#         n_errors = torch.nonzero(torch.abs(labels.long() - predicted)).size(0)
#         total += predicted.numel()
#         correct += predicted.numel() - n_errors
#         print('Accuracy {:.2f}%'.format(100*correct/total))

    print("Loss epoch {}: {}, took {}s".format(epoch, running_loss,time.time()-start_time))
    loss_array.append(running_loss)
    learning_rate_array.append(learning_rate)
print('...Training finished')


Epoch: 0
Accuracy 0.00%
Accuracy 0.00%
Accuracy 0.00%


In [ ]:
#DYNAMIC CHANGES:
NR_EPOCHS = 20
learning_rate = 0.0001
momentum = 0.9
optimizer = optim.SGD(test_net.parameters(), lr=learning_rate, momentum=momentum)
print("done")

In [ ]:
#EVALUATION
print("Expected loss:{}, last loss:{}".format((len(dataset)-split)*np.log(3),loss_array[-1]))
print("Batch size:", BATCH_SIZE)
print("Sequence length:",TIMESTEPS)
print("Total epochs learnt:", len(loss_array))
#plt.plot(learning_rate_array)
#plt.plot(loss_array)
### define and execute testing function
def test_all_preds(model):
    n_batches_test = len(test_loader)

    #Time for printing
    testing_start_time = time.time()

    print('Start testing...')
    correct = 0 
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            
            data_in = inputs['flows'].to(device)
            labels = labels.to(device)

#             if not labels.size()[0] == BATCH_SIZE:
#                 # skip uncompleted batch size NN is fixed to BATCHSIZE
#                 continue
            outputs = model(data_in)
            for item in labels:
                labels = torch.tensor([[item]])
#             print("Out:", len(outputs), outputs.size())
#             print("Labels:", len(labels), labels.size())
            _, predicted = torch.max(outputs.data, 1)
#             print('predicted:',len(predicted),predicted.size())
            n_errors = torch.nonzero(torch.abs(labels.long() - predicted)).size(0)
            total += predicted.numel()
            # print('predicted',predicted)
            correct += predicted.numel() - n_errors
            # print('labels',labels)
    print('Accuracy {:.2f}%'.format(100*correct/total))
    print('...testing finished')
print("Definition Test predictions done")
test_all_preds(test_net)


In [ ]:
parameters_fc1 = list(test_net.fc1.parameters())
bias_fc1 = parameters_fc1[1].detach().numpy()
weights_fc1 = parameters_fc1[0].detach().numpy().flatten()

parameters_conv1 = list(test_net.conv1.parameters())
bias_conv1 = parameters_conv1[1].detach().numpy()
weights_conv1 = parameters_conv1[0].detach().numpy().flatten()

weights_fc1_m = np.mean(weights_fc1-weights_fc1_init)
weights_fc1_v = np.var(weights_fc1-weights_fc1_init)

print("FC1: Mean",np.mean(weights_fc1_init),"Variance",np.var(weights_fc1_init))
print("FC1: Mean",np.mean(weights_fc1),"Var",np.var(weights_fc1))
print("FC1 Diff: Var",np.var(weights_fc1_init-weights_fc1))
#plt.figure()
#plt.plot(range(len(weights_conv1)),weights_conv1_init,range(len(weights_conv1)),weights_conv1)
#plt.figure()
#plt.plot(range(len(bias_conv1)),bias_conv1_init,range(len(bias_conv1)),bias_conv1)

##FC layers
#plt.figure()
#plt.plot(range(len(weights_conv1)),weights_conv1_init,range(len(weights_conv1)),weights_conv1)
#plt.figure()
#plt.plot(range(len(weights_fc1)),weights_fc1_init,range(len(weights_fc1)),weights_fc1)

plt.figure()
plt.title("Difference in FC-Layers")
plt.plot(range(len(weights_fc1)),weights_fc1_init-weights_fc1)

plt.figure()
plt.title("Difference in Conv-Layers")
plt.plot(range(len(weights_conv1)),weights_conv1_init-weights_conv1)#,range(len(weights_conv5)),weights_conv5)

#plt.figure()
#plt.plot(weights_lstm2_init-weights_lstm2)


In [ ]:
plt.figure()
plt.title("Difference in FC-Layers")
plt.plot(range(len(weights_fc1)),weights_fc1_init-weights_fc1)

plt.draw()
#plt.show()
print("done")

### Expected loss for untrained set with Cross Entropy:
k = number of classes
N = number of labeled data in dataset
loss_per_prediction = -log(1/k) = log(k)
total_loss = sum(log(k)) = N*log(k)

In [ ]:
plt.clf()
plot_path = "/home/matthias/Desktop/"
loss_str = "{0:.3f}".format(loss_array[-1])
information = "flows_L" + loss_str + "_timesteps{}_nrsequences{}_batch{}".format(TIMESTEPS,dataset_size,BATCH_SIZE)
plt.xlabel('Epochs')
plt.ylabel('Loss (Cross Entropy)')
plt.plot(loss_array)
plt.title(information)
plt.savefig(plot_path + information + ".eps")
plt.clf()
plt.xlabel('Epochs')
plt.ylabel('Learning rate')
plt.plot(learning_rate_array)
plt.title(information)
plt.savefig(plot_path + information + "LR.eps")

In [ ]:
#torch.save(test_net.state_dict(), '/mnt/DATA/HIWI/IBT/saved_models/cnn_fixed_timesteps/cnn_10_steps_fixed')